## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [0]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

170500096/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3)


In [4]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [5]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3,3), input_shape=(32,32,3),activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

In [6]:
#'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [7]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
Total params: 1,024
Trainable params: 960
Non-trainable params: 64
_________________________________________________________________


In [0]:

#卷積組合
classifier.add(Convolution2D(32, (3,3), input_shape=(15,15,32),activation='relu'))
classifier.add(BatchNormalization())

In [0]:
#flatten
classifier.add(Flatten())

In [11]:
#FC
classifier.add(Dense(output_dim=100,activation='relu')) #output_dim=100,activation=relu

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
  """Entry point for launching an IPython kernel.


In [12]:
#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  """Entry point for launching an IPython kernel.


In [17]:
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)  # 100, 100

Epoch 1/100
50000/50000 [==============================] - 84s 2ms/step - loss: 0.0632 - acc: 0.9790
Epoch 2/100
50000/50000 [==============================] - 84s 2ms/step - loss: 0.0589 - acc: 0.9801
Epoch 3/100
50000/50000 [==============================] - 84s 2ms/step - loss: 0.0477 - acc: 0.9831
Epoch 4/100
50000/50000 [==============================] - 83s 2ms/step - loss: 0.0397 - acc: 0.9862
Epoch 5/100
50000/50000 [==============================] - 83s 2ms/step - loss: 0.0423 - acc: 0.9854
Epoch 6/100
50000/50000 [==============================] - 83s 2ms/step - loss: 0.0531 - acc: 0.9819
Epoch 7/100
50000/50000 [==============================] - 84s 2ms/step - loss: 0.0427 - acc: 0.9861
Epoch 8/100
50000/50000 [==============================] - 84s 2ms/step - loss: 0.0355 - acc: 0.9878
Epoch 9/100
50000/50000 [==============================] - 84s 2ms/step - loss: 0.0361 - acc: 0.9882
Epoch 10/100
50000/50000 [==============================] - 85s 2ms/step - loss: 0.0423 - a

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [18]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[3.4578450e-03, 5.6662787e-14, 1.0813290e-07, 9.9653530e-01,
        6.7034398e-06, 3.4587195e-16, 2.3080404e-17, 3.6272904e-10,
        1.5802087e-14, 1.4822052e-18]], dtype=float32)